In [1]:
import glob
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
# folder_path = "DATA"
# csv_files = glob.glob(folder_path + "/*.csv")
# print(csv_files)
os.environ["DB_URL"]

'sqlite:///DATA/plm_updated.db'

In [ ]:
import os
from sqlalchemy import create_engine, inspect
ENGINE = create_engine(os.environ["DB_URL"])
inspector = inspect(ENGINE)

In [3]:
table_names = inspector.get_table_names()
for name in table_names:
    columns = inspector.get_columns(name)
    print(name, columns)

RecipeExplosion [{'name': 'CUCSpecCode', 'type': BIGINT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCRecipeCode', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCMaterialCode', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCPlantCode', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCPlantDescription', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCPlantOwnershipFlag', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCPlantCountryCode', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCPlantCountryName', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCPlantBUCode', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key': 0}, {'name': 'CUCPlantBUName', 'type': TEXT(), 'nullable': True, 'default': None, 'primary_key'

In [14]:
import pandas as pd
query = """SELECT * FROM RecipeExplosion LIMIT 5"""
df = pd.read_sql_query(query, ENGINE)
# print(df)
df

,CUCSpecCode,CUCRecipeCode,CUCMaterialCode,CUCPlantCode,CUCPlantDescription,CUCPlantOwnershipFlag,CUCPlantCountryCode,CUCPlantCountryName,CUCPlantBUCode,CUCPlantBUName,INGSpecCode,INGMaterialCode,INGPlantCode,Ing2CUC_PercentageContribution,Ing2CUC_QuantityContribution
0,200000082485,200000082485/002/003,9144595_S,A852,IT Caivano,UL,IT,Italy,EU,Europe,100000066661,376054_S,A852,0.000110,0.001100
1,200000251809,200000251809/006/001,68640589_F,UTE,HUL-NESA PP (Oral),UL,IN,India,SA,South Asia,100000043581,11060121_F,UTE,2.000000,2000.000000
2,200000094250,200000094250/002/000,8736480_S,A976,UBF-CH-SU Thayngen,UL,CH,Switzerland,EU,Europe,100000083120,3711914_S,A502,2.425476,0.436586
3,200000007387,200000007387/002/001,20231424_U,7001,Minburi HPC,UL,TH,Thailand,GCEA,GCEA,100000005644,66810211_U,7001,4.348000,43.480000
4,200000088676,200000088676/002/003,67143582_C,3165,UBI Fca Suape HC,UL,BR,Brazil,LATAM,Latin America,100000012619,563546_C,3165,0.010000,0.100000


In [44]:
import pandas as pd
#Total rows 8812
# ING    6605
# CUC    2207
query = f"""SELECT s.specdescription
FROM   recipeexplosion re
       JOIN specifications s
         ON re.ingspeccode = s.speccode
WHERE  Upper(s.specdescription) LIKE Upper('%TEA%')
LIMIT  200;"""
df = pd.read_sql_query(query, ENGINE)
# df
print(df)

                                     SpecDescription
0    Acrylates/Steareth-20 Methacrylate Crosspolymer
1    Acrylates/Steareth-20 Methacrylate Crosspolymer
2    Acrylates/Steareth-20 Methacrylate Crosspolymer
3    Acrylates/Steareth-20 Methacrylate Crosspolymer
4    Acrylates/Steareth-20 Methacrylate Crosspolymer
..                                               ...
195                         Tea Black Leaf Blend 425
196                         Tea Black Leaf Blend 425
197                         Tea Black Leaf Blend 425
198                         Tea Black Leaf Blend 425
199                         Tea Black Leaf Blend 425

[200 rows x 1 columns]


In [29]:
df.columns

Index(['SpecCode', 'SpecDescription', 'SpecStatusDescription', 'SpecGroupCode',
       'SpecTypeDescription', 'AuthorisationGroupCode', 'SpecTypeCode'],
      dtype='object')

In [ ]:
recp_explo_file = "DATA\\RecipeExplosionForLLM_20250731_v2.csv"
spec_file = "DATA\\SpecsForLLM_20250731_v2.csv"
df_spec = pd.read_csv(spec_file)
df_spec = df_spec[df_spec['SpecCode'].astype(str).str.match(r'^\d+$')]
dtype_mapping = {
    'SpecCode': 'int64' 
}
df_spec = df_spec.astype(dtype_mapping)

In [117]:
df_spec.columns

Index(['SpecCode', 'SpecDescription', 'SpecStatusDescription', 'SpecGroupCode',
       'SpecTypeDescription', 'AuthorisationGroupCode', 'SpecTypeCode'],
      dtype='object')

In [121]:
len(df_spec)
valid_spec_codes = df["SpecCode"]
_is_valid = df_spec['SpecCode'].isin(valid_spec_codes)
df_spec = df_spec[_is_valid].reset_index(drop=True)
print(len(df_spec))
df_spec.dtypes

8812


SpecCode                   int64
SpecDescription           object
SpecStatusDescription     object
SpecGroupCode             object
SpecTypeDescription       object
AuthorisationGroupCode    object
SpecTypeCode              object
dtype: object

In [ ]:
df_spec.to_csv("DATA\\SpecsForLLM_20250802.csv", index=False)

In [107]:
df_explo =  pd.read_csv(recp_explo_file)
columns_to_remove = ['DUSpecCode', 'DURecipeCode', 'DUMaterialCode', 'DUPlantCode', 'DUPlantDescription', 'DUPlantOwnershipFlag', 'DUPlantCountryCode',
       'DUPlantCountryName', 'DUPlantBUCode', 'DUPlantBUName', 'ExplosionRecipePathValue', 'ExplosionSpecPathValue', 'ExplosionMaterialPathValue']
df_explo_new = df_explo.drop(columns=columns_to_remove)
dtype_mapping = {
    'CUCSpecCode': 'str',
    'INGSpecCode': 'str' 
}
df_explo_new = df_explo_new.astype(dtype_mapping)
df_explo_new.dtypes

C:\Users\ShyamSunderIyer\AppData\Local\Temp\ipykernel_32468\3099180847.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_explo =  pd.read_csv(recp_explo_file)


CUCSpecCode                        object
CUCRecipeCode                      object
CUCMaterialCode                    object
CUCPlantCode                       object
CUCPlantDescription                object
CUCPlantOwnershipFlag              object
CUCPlantCountryCode                object
CUCPlantCountryName                object
CUCPlantBUCode                     object
CUCPlantBUName                     object
INGSpecCode                        object
INGMaterialCode                    object
INGPlantCode                       object
Ing2CUC_PercentageContribution    float64
Ing2CUC_QuantityContribution      float64
dtype: object

In [ ]:
filtered_df_spec = df_spec[df_spec['SpecCode'].isin(df["SpecCode"])].reset_index(drop=True)


In [ ]:
dtype_mapping = {
    'SpecCode': 'int64',
    'SpecDescription': 'str',
    'SpecStatusDescription': 'str',
    'SpecGroupCode': 'str',
    'SpecTypeDescription': 'str',
    'AuthorisationGroupCode': 'str',
    'SpecTypeCode': 'str' 
}
filtered_df_spec = filtered_df_spec.astype(dtype_mapping)
filtered_df_spec.to_csv("DATA\\SpecsForLLM_20250802.csv", index=False)

In [57]:
dtype_mapping = {
    'SpecCode': 'int64',
    'SpecDescription': 'str',
    'SpecStatusDescription': 'str',
    'SpecGroupCode': 'str',
    'SpecTypeDescription': 'str',
    'AuthorisationGroupCode': 'str',
    'SpecTypeCode': 'str' 
}
filtered_df_spec = filtered_df_spec.astype(dtype_mapping)
# filtered_df_spec_converted.info()

In [100]:
filtered_df_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8812 entries, 0 to 8811
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   SpecCode                8812 non-null   int64 
 1   SpecDescription         8812 non-null   object
 2   SpecStatusDescription   8812 non-null   object
 3   SpecGroupCode           8812 non-null   object
 4   SpecTypeDescription     8812 non-null   object
 5   AuthorisationGroupCode  8812 non-null   object
 6   SpecTypeCode            8812 non-null   object
dtypes: int64(1), object(6)
memory usage: 482.0+ KB


In [38]:
df_explo.columns

Index(['DUSpecCode', 'DURecipeCode', 'DUMaterialCode', 'DUPlantCode',
       'DUPlantDescription', 'DUPlantOwnershipFlag', 'DUPlantCountryCode',
       'DUPlantCountryName', 'DUPlantBUCode', 'DUPlantBUName', 'CUCSpecCode',
       'CUCRecipeCode', 'CUCMaterialCode', 'CUCPlantCode',
       'CUCPlantDescription', 'CUCPlantOwnershipFlag', 'CUCPlantCountryCode',
       'CUCPlantCountryName', 'CUCPlantBUCode', 'CUCPlantBUName',
       'INGSpecCode', 'INGMaterialCode', 'INGPlantCode',
       'Ing2CUC_PercentageContribution', 'Ing2CUC_QuantityContribution',
       'ExplosionRecipePathValue', 'ExplosionSpecPathValue',
       'ExplosionMaterialPathValue'],
      dtype='object')

In [40]:
columns_to_remove = ['DUSpecCode', 'DURecipeCode', 'DUMaterialCode', 'DUPlantCode', 'DUPlantDescription', 'DUPlantOwnershipFlag', 'DUPlantCountryCode',
       'DUPlantCountryName', 'DUPlantBUCode', 'DUPlantBUName', 'ExplosionRecipePathValue', 'ExplosionSpecPathValue', 'ExplosionMaterialPathValue']
df_explo_new = df_explo.drop(columns=columns_to_remove)
df_explo_new.head()

,CUCSpecCode,CUCRecipeCode,CUCMaterialCode,CUCPlantCode,CUCPlantDescription,CUCPlantOwnershipFlag,CUCPlantCountryCode,CUCPlantCountryName,CUCPlantBUCode,CUCPlantBUName,INGSpecCode,INGMaterialCode,INGPlantCode,Ing2CUC_PercentageContribution,Ing2CUC_QuantityContribution
0,200000240332,200000240332/001/002,68276743_C,3614,Cuernavaca HPC 3614,UL,MX,Mexico,LATAM,Latin America,100000010048,935180_C,3614,0.31000,3.1000
1,200000139049,200000139049/011/001,20212770_U,8005,Conghua Foods,UL,CN,China,GCEA,GCEA,100000038019,63131312_U,8005,18.00000,72.0000
2,200000251449,200000251449/003/001,62678319_C,3087,Cochabamba,UL,BO,Bolivia,LATAM,Latin America,100000001064,956683_C,3087,0.10000,100.0000
3,200000082485,200000082485/002/003,9144595_S,A852,IT Caivano,UL,IT,Italy,EU,Europe,100000066661,376054_S,A852,0.00011,0.0011
4,200000272665,200000272665/006/000,69990242_U,9003,Cikarang Skin,UL,ID,Indonesia,GCEA,GCEA,100000001437,67701482_U,9003,0.04320,0.4320


In [99]:
# len(df_explo_new.CUCSpecCode.unique()), len(df_explo_new.INGSpecCode.unique())
df_explo_new.dtypes

CUCSpecCode                         int64
CUCRecipeCode                      object
CUCMaterialCode                    object
CUCPlantCode                       object
CUCPlantDescription                object
CUCPlantOwnershipFlag              object
CUCPlantCountryCode                object
CUCPlantCountryName                object
CUCPlantBUCode                     object
CUCPlantBUName                     object
INGSpecCode                         int64
INGMaterialCode                    object
INGPlantCode                       object
Ing2CUC_PercentageContribution    float64
Ing2CUC_QuantityContribution      float64
dtype: object

In [63]:
# len(set(filtered_df_spec.SpecCode)) == len(filtered_df_spec.SpecCode)
# len(set(df_explo_new.INGSpecCode))
len(df_explo_new)


1000000

In [66]:
len(set(filtered_df_spec.SpecCode).intersection(set(df_explo_new.INGSpecCode))) #6578 2205

6578

In [68]:
valid_spec_codes = filtered_df_spec['SpecCode']
cuc_is_valid = df_explo_new['CUCSpecCode'].isin(valid_spec_codes)
ing_is_valid = df_explo_new['INGSpecCode'].isin(valid_spec_codes)
combined_mask = cuc_is_valid & ing_is_valid
df_explo_filtered = df_explo_new[combined_mask]
df_explo_filtered = df_explo_filtered.reset_index(drop=True)

In [70]:
df_explo_filtered.head()

,CUCSpecCode,CUCRecipeCode,CUCMaterialCode,CUCPlantCode,CUCPlantDescription,CUCPlantOwnershipFlag,CUCPlantCountryCode,CUCPlantCountryName,CUCPlantBUCode,CUCPlantBUName,INGSpecCode,INGMaterialCode,INGPlantCode,Ing2CUC_PercentageContribution,Ing2CUC_QuantityContribution
0,200000082485,200000082485/002/003,9144595_S,A852,IT Caivano,UL,IT,Italy,EU,Europe,100000066661,376054_S,A852,0.000110,0.001100
1,200000251809,200000251809/006/001,68640589_F,UTE,HUL-NESA PP (Oral),UL,IN,India,SA,South Asia,100000043581,11060121_F,UTE,2.000000,2000.000000
2,200000094250,200000094250/002/000,8736480_S,A976,UBF-CH-SU Thayngen,UL,CH,Switzerland,EU,Europe,100000083120,3711914_S,A502,2.425476,0.436586
3,200000007387,200000007387/002/001,20231424_U,7001,Minburi HPC,UL,TH,Thailand,GCEA,GCEA,100000005644,66810211_U,7001,4.348000,43.480000
4,200000088676,200000088676/002/003,67143582_C,3165,UBI Fca Suape HC,UL,BR,Brazil,LATAM,Latin America,100000012619,563546_C,3165,0.010000,0.100000


In [79]:
filtered_df_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8812 entries, 0 to 8811
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   SpecCode                8812 non-null   int64 
 1   SpecDescription         8812 non-null   object
 2   SpecStatusDescription   8812 non-null   object
 3   SpecGroupCode           8812 non-null   object
 4   SpecTypeDescription     8812 non-null   object
 5   AuthorisationGroupCode  8812 non-null   object
 6   SpecTypeCode            8812 non-null   object
dtypes: int64(1), object(6)
memory usage: 482.0+ KB


In [72]:
tables = {
    "Specifications": filtered_df_spec,
    "RecipeExplosion": df_explo_filtered
}

In [ ]:
from data_processing.vectors import build_retriever_system
retrievers = build_retriever_system(ENGINE, data)

In [5]:
rules_retriever, schema_desc_retriever, table_info_retriever, table_scheme_retriever, column_value_retriever = retrievers
query = "Give me all recepes with jam and butter in it"

In [6]:
nodes = rules_retriever.retrieve(query)
text = "### Business Rules & Formulas:\n" + "\n\n".join([n.get_content() for n in nodes])
print(text)

### Business Rules & Formulas:
Table Relationship: Connects a final product recipe to its list of ingredients (Bill of Materials). (recipes.RECIPEKEY -> line_items.RECIPEKEY)

Table Relationship: Connects an ingredient in a recipe to its detailed chemical composition/formula. (line_items.SPECIFICATIONNR -> specifications.SPECIFICATIONNR)

Golden Formula 'Absolute Component Mass Calculation': To calculate this, use the SQL formula `SUM(li.ITEMQUANTITY * s.AVG_VAL / 100.0)`. Notes: Ensure that specifications.UOM is '%' before applying. If it is 'PPB' or 'PPM', convert to percent first.


In [7]:
nodes = schema_desc_retriever.retrieve(query)
text = "### Relevant Table & Column and their Schemas:\n" + "\n\n".join([n.get_content() for n in nodes])
print(text)

### Relevant Table & Column and their Schemas:
Table: line_items. Description: The Bill of Materials (BOM). It lists all ingredients and their quantities for a specific recipe.
Columns:

  - ITEMNR (INTEGER)
  - ITEMTYPE (TEXT)
  - ITEMTYPE_DESCR (TEXT)
  - SPECIFICATIONNR (INTEGER)
  - SPECDESCRIPTION (TEXT)
  - ITEMMATERIAL (FLOAT)
  - MATERIALDESCRIPTION (FLOAT)
  - ITEMQUANTITY (FLOAT)
  - ITEMUOM (TEXT)
  - COMPTYPE (TEXT)
  - COMPTYPE_DESCR (TEXT)
  - ITEMPMLCOLOR (TEXT)
  - SOURCEPLANT (FLOAT)
  - YIELD (FLOAT)
  - MASS_PER_POIN_PERCENT (FLOAT)
  - MANUAL_PRICE (FLOAT)
  - MANUAL_PRICE_CURRENCY (TEXT)
  - QTYLOW (FLOAT)
  - QTYHIGH (FLOAT)
  - EXCEPTIONVALUE (FLOAT)
  - SCRAPFACTOR (FLOAT)
  - ABSOLUTEMASS (FLOAT)
  - OVERRUNSCRAP (FLOAT)
  - ZEROSIGN (FLOAT)
  - CALCULATEQTY (TEXT)
  - REQACTIVE (FLOAT)
  - PERCENTAGEACTIVE (FLOAT)
  - ACTCOMPIDENTIFIER (FLOAT)
  - NOTE (FLOAT)
  - RESONFORUSE (TEXT)
  - OVERALLPMLCOLOR (TEXT)
  - PROPINTVALUE (TEXT)
  - ISIDPARTNO (TEXT)
  - T

In [8]:
nodes = table_info_retriever.retrieve(query)
text = "### Relevant Table & Column and their Schemas:\n" + "\n\n".join([n.get_content() for n in nodes])
print(text)

### Relevant Table & Column and their Schemas:







In [9]:
nodes = table_scheme_retriever.retrieve(query)
text = "### Relevant Table & Column and their Schemas:\n" + "\n\n".join([n.get_content() for n in nodes])
print(text)

### Relevant Table & Column and their Schemas:







In [10]:
nodes = column_value_retriever.retrieve(query)
text = "### Relevant Column and their sample Values:\n" + "\n\n".join([n.get_content() for n in nodes])
print(text)
            

### Relevant Column and their sample Values:
The commercial or common name of the final product. Use this for filtering by name. Example values for column 'RECIPEDESCRIPTION' in table 'recipes': Knorrox Beef Sodium Reduction, CAS T&C Potato & Leek, BLEND ROYCO BEEF CUBE, Knorr Italian White Sauce without Annato & Cheese Powder, Kissan Tomato Paste VIP, Kissan Tomato Puree VIP, ROYCO MCHUZI MIX BEEF BLEND, KNORR soup chicken flavor bdz 400g CUC, KNORR BASE CHICKEN 095 DARWIN, KNORR CHEF'S DAL MASALA with MAXAROME

The human-readable name of the ingredient used in the recipe. Example values for column 'SPECDESCRIPTION' in table 'line_items': Knx Beef Cube, Fat Krema, Fat hard Vemor 50, Caramel Liquid ammonia E150c HK EBC 48000, Col Ponceau 4R PSU 0730 Hexacol, Potable Water, Salt Blended (MEDIUM) tuff Bags, Monosodiumglutamate Regular 24-60 mesh, I & G Nucleotides Kosher Parev 20kg cart, Ground Thyme

Example values for column 'RECIPEVERSIONLIST' in table 'recipes': []

Example values fo

In [27]:
import sqlite3
from typing import Union
import pandas as pd
def execute_sql(query: str) -> Union[pd.DataFrame, str]:
    """Executes a SQL query against the SQLite database."""
    if not query:
        return "Cannot execute an empty query."
    try:
        DB_URL = os.path.join(os.getcwd(), 'uploaded_data.db')
        with sqlite3.connect(DB_URL) as conn:
            conn: sqlite3.Connection
            df: pd.DataFrame = pd.read_sql_query(query, conn)
            if df.empty:
                return "No data found matching your criteria."
            return df
    except Exception as e:
        return f"SQL Execution Error: {e} on query:\n---\n{query}\n---"

In [ ]:
sql_query = """SELECT * FROM PML_OUTPUT_recipes LIMIT 5;"""
df = execute_sql(sql_query)
print(df)

              RECIPEKEY                                  RECIPEDESCRIPTION  \
0  200000029428/005/000                      Knorrox Beef Sodium Reduction   
1  200000029601/003/000                              CAS T&C Potato & Leek   
2  200000043297/005/000                              BLEND ROYCO BEEF CUBE   
3  200000049981/003/000  Knorr Italian White Sauce without Annato & Che...   
4  200000071789/000/000                            Kissan Tomato Paste VIP   

       RECIPETYPE  RECIPESTATUS RECIPEAUTHGROUP HEADERMATERIAL RECIPEPLANT  \
0  Z_EXPERIMENTAL           610            ZSVR           None        None   
1  Z_EXPERIMENTAL           610            ZSVR           None        None   
2  Z_EXPERIMENTAL           610            ZSVR           None        None   
3  Z_EXPERIMENTAL           510            ZSVR           None        None   
4  Z_EXPERIMENTAL           510            ZSVR           None        None   

  PREFERREDRECIPE PREFERREDVALIDFROM PREFERREDVALIDTO       CR

In [35]:
import re
name = "PML_OUTPUT/line_items.csv"
# re.sub(r'\W+', '_', str(name)).strip('_') 
import os
os.path.splitext(name)

('PML_OUTPUT/line_items', '.csv')

In [12]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embeddings = embed_model.get_text_embedding("This is a test sentence.")
print(embeddings)

c:\Users\ShyamSunderIyer\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[-0.06439797580242157, 0.0290534570813179, 0.008177057839930058, -0.006123151630163193, 0.011226010508835316, 0.006352214142680168, 0.07471279054880142, 0.015220386907458305, 0.03067406453192234, -0.035817183554172516, -0.020257389172911644, -0.029482632875442505, 0.02826531045138836, 0.03605448454618454, 0.004387761000543833, -0.016454221680760384, 0.0022171312011778355, -0.06411594897508621, -0.06320693343877792, 0.028692586347460747, 0.07384111732244492, -0.007364855147898197, -0.009883039630949497, -0.013002180494368076, 0.0011464889394119382, 0.0426669679582119, 0.022866519168019295, -0.03134549409151077, -0.019507452845573425, -0.08184880018234253, 0.011755703948438168, -0.033044617623090744, 0.02787681110203266, -0.005916205700486898, 0.02138533815741539, 0.017969202250242233, -0.06019717827439308, -0.017807118594646454, -0.045378461480140686, -0.021133672446012497, 0.05267377197742462, 0.011488471180200577, -0.028552083298563957, 0.004141320940107107, 0.02155274897813797, 0.021

In [26]:
import json
with open("schema_description.json", "r") as file:
    data = json.load(file)
print("File loaded successfully!")
print(data)

File loaded successfully!
{'title': 'Business Schema for Recipe & Manufacturing Database', 'description': 'This file is the single source of truth for the AI agent, defining business logic for accurate SQL queries.', 'relationships': [{'from_table': 'RecipeExplosion', 'from_column': 'CUCSpecCode', 'to_table': 'Specifications', 'to_column': 'SpecCode', 'description': 'Connects a recipe from the RecipeExplosion table to its detailed description and metadata in the Specifications table.'}, {'from_table': 'RecipeExplosion', 'from_column': 'INGSpecCode', 'to_table': 'Specifications', 'to_column': 'SpecCode', 'description': 'Connects an ingredient from the RecipeExplosion table to its detailed description and metadata in the Specifications table.'}], 'golden_formulas': [{'name': 'Absolute Component Mass Calculation', 'description': "To calculate the absolute mass of a specific substance (e.g., 'Salt') in a final recipe, you MUST use this formula. This requires joining all three tables.", 'fo